In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [3]:
def Change_To_Num(Val):
    Result = Val.text.replace(',','')
    Result = Result[:Result.find(' ')]
    Result = pd.to_numeric(Result)
    Val = Val.text
    Val = Val[-1:]
    if Val=='M':
        Result = Result*1000000
    elif Val=='B':
        Result = Result*1000000000
    else:
        Result = Result
    return Result

In [4]:
Driver = webdriver.Chrome(executable_path=r"D:\chromedriver.exe")
Driver.get("http://www.tsetmc.com/Loader.aspx?ParTree=15")

In [5]:
def Find_Element(Srt):
    Result = Val.text.replace(',','')
    Result = pd.to_numeric(Result)
    return Result

In [7]:
Symbol_List = ['ونفت','شپنا','امید','فملی']
#Symbol_List = Symbols['symbol'].unique()
# Symbol_List = ['ذوب','ونفت','شپنا','امید','خودرو']

for SL in Symbol_List:
    try:
        #search
        Search = Driver.find_element_by_css_selector("#search")
        Search.send_keys(Keys.ENTER)
        Search_Key = Driver.find_element_by_css_selector("#SearchKey")
        Search_Key.send_keys(SL)
        Search_Key.send_keys(Keys.ENTER)
        time.sleep(5)
        Search_Result = Driver.find_element_by_css_selector("#SearchResult > div > div.content > table > tbody > tr:nth-child(1) > td:nth-child(1) > a")
        Search_Result.click()
        #MainBox > div.header.bigheader  نام 
        MainBox = Driver.find_element_by_css_selector('#MainBox > div')
        MainBox = MainBox.text
        MainBox = MainBox[:MainBox.find(" (")]
        print("MainBox ---> ",MainBox)   

        #d02   قیمت لحظه ای و تغییرات
        time.sleep(10)
        Pl = Driver.find_element_by_css_selector('#d02')
        Pl = Pl.text

        Price = Pl[:Pl.find("  ")].replace(',','')
        Price = pd.to_numeric(Price)
        print("Price ---> ",Price)   #trim

        Sign_Change = Pl[Pl.find('%')+1]
        Percent_Change = Pl[Pl.find('[')+1:Pl.find('%')].replace(',','')
        Percent_Change = pd.to_numeric(Percent_Change)/100

        if Sign_Change == ']':
            Changes = Pl[Pl.find('  ')+1:Pl.find('[')].replace(',','')
            Changes = pd.to_numeric(Changes)
        elif  Sign_Change == '-':
            Changes = Pl[Pl.find('(')+1:Pl.find(')')].replace(',','')
            Changes = -(pd.to_numeric(Changes))
            Percent_Change = -abs(Percent_Change)

        print("Changes ---> ",Changes)
        print("Percent_Change ---> ",Percent_Change)
        #d03 > span:nth-child(2)  تغییرات پایانی و درصد تغییرات
        Pcc_Pcp = Driver.find_element_by_css_selector('#d03 > span:nth-child(2)')
        Pcc_Pcp = Pcc_Pcp.text

        Sign_Change = Pcc_Pcp[Pcc_Pcp.find('%')+1]
        Pcp = Pcc_Pcp[Pcc_Pcp.find('[')+1:Pcc_Pcp.find('%')].replace(',','')
        Pcp = pd.to_numeric(Pcp)/100

        if Sign_Change == ']':
            Pcc = Pcc_Pcp[:Pcc_Pcp.find('  ')].replace(',','')
            Pcc = pd.to_numeric(Pcc)
        elif  Sign_Change == '-':
            Pcc = Pcc_Pcp[Pcc_Pcp.find('(')+1:Pcc_Pcp.find(')')].replace(',','')
            Pcc = -(pd.to_numeric(Pcc))
            Pcp = -abs(Pcp)

        print('Pcc ---> ',Pcc)
        print('Pcp ---> ',Pcp)
        #d03 > span:nth-child(1) قیمت پایانی
        Pc = Driver.find_element_by_css_selector('#d03 > span:nth-child(1)')
        print('قیمت پایانی ---> ',Change_To_Num(Pc))
        #d04  قیمت اولین
        Pf = Driver.find_element_by_css_selector('#d04')
        print('قیمت اولین ---> ',Change_To_Num(Pf))    
        #d05  قیمت دیروز
        Py = Driver.find_element_by_css_selector('#d05')
        print('قیمت دیروز ---> ',Change_To_Num(Py))
        #d08    تعداد معاملات
        Tno = Driver.find_element_by_css_selector('#d08')
        print('تعداد معاملات ---> ',Change_To_Num(Tno))
        #d09    حجم معاملات
        Tvol = Driver.find_element_by_css_selector('#d09')
        Tvol =  Change_To_Num(Tvol)
        print('حجم معاملات ---> ', Tvol)
        #d10    ارزش معاملات
        Tval = Driver.find_element_by_css_selector('#d10')
        Tval = Change_To_Num(Tval)
        print('ارزش معاملات ---> ', Tval)
        #d02    ارزش بازار
        Mv = Driver.find_element_by_css_selector('#d11')   #'230,084.738 B'
        Mv = Change_To_Num(Mv)
        print('ارزش بازار ---> ', Mv)
        #TopBox > div.box2.zi1 > div:nth-child(6) > table > tbody > tr:nth-child(1) > td:nth-child(2)
        #  EPS
        EPS = Driver.find_element_by_css_selector('#TopBox > div.box2.zi1 > div:nth-child(6) > table > tbody > tr:nth-child(1) > td:nth-child(2)')
        print('EPS ---> ',Change_To_Num(EPS))
        #d12       
        #  P/E
        Pe = Driver.find_element_by_css_selector('#d12')
        print('P/E ---> ',Change_To_Num(Pe))
        #TopBox > div.box2.zi1 > div:nth-child(6) > table > tbody > tr:nth-child(1) > td:nth-child(6)  PEگروه
        #  P/E گروه
        Pe_Group = Driver.find_element_by_css_selector('#TopBox > div.box2.zi1 > div:nth-child(6) > table > tbody > tr:nth-child(1) > td:nth-child(6)')
        print('P/E گروه ---> ',Change_To_Num(Pe_Group))
        #TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(1) > td:nth-child(2) > div   تعداد سهام
        # تعداد سهام
        Z = Driver.find_element_by_css_selector('#TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(1) > td:nth-child(2) > div')
        print('تعداد سهام ---> ',Change_To_Num(Z))
        #TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(2) > td:nth-child(2)   حجم مبنا
        #   حجم مبنا
        Bvol = Driver.find_element_by_css_selector('#TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(2) > td:nth-child(2)')
        print('حجم مبنا ---> ',Change_To_Num(Bvol))
        #TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(3) > td:nth-child(2)   سهام شناور
        #    سهام شناور
        Floating_Stock = Driver.find_element_by_css_selector('#TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(3) > td:nth-child(2)')
        Floating_Stock.text  
        print('سهام شناور ---> ',Floating_Stock)
        #TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(4) > td:nth-child(2)   میانگین حجم ماه
        # میانگین حجم ماه
        Avrage_Volume_Per_Month = Driver.find_element_by_css_selector('#TopBox > div.box2.zi1 > div:nth-child(4) > table > tbody > tr:nth-child(4) > td:nth-child(2)')
        print('میانگین حجم ماه ---> ',Change_To_Num(Avrage_Volume_Per_Month))
        #d06   بیشترین قیمت روز
        Pmax = Driver.find_element_by_css_selector('#d06')
        Pmax = Change_To_Num(Pmax)
        print('بیشترین قیمت روز ---> ',Pmax)
        #d07   کمترین قیمت روز
        Pmin = Driver.find_element_by_css_selector('#d07')
        Pmin = Change_To_Num(Pmin)
        print('کمترین قیمت روز ---> ',Pmin)
        #PRange1    حداکثر قیمت مجاز روز
        Tmax = Driver.find_element_by_css_selector('#PRange1')
        Tmax = Change_To_Num(Tmax)
        print('حداکثر قیمت مجاز روز ---> ',Tmax)
        #PRange2    حداقل قیمت مجاز روز
        Tmin = Driver.find_element_by_css_selector('#PRange2')
        Tmin = Change_To_Num(Tmin)
        print('حداقل قیمت مجاز روز ---> ',Tmin)
        #e0 > div:nth-child(1)  حجم خرید حقیقی
        Buy_I_Volume = Driver.find_element_by_css_selector('#e0 > div:nth-child(1)')
        Buy_I_Volume = Change_To_Num(Buy_I_Volume)
        print('حجم خرید حقیقی ---> ',Buy_I_Volume)
        #e1 > div:nth-child(1)  حجم خرید حقوقی
        Sell_I_Volume = Driver.find_element_by_css_selector('#e1 > div:nth-child(1)')
        Sell_I_Volume = Change_To_Num(Sell_I_Volume)
        print('حجم خرید حقوقی ---> ',Sell_I_Volume)
        #e3 > div:nth-child(1)  حجم فروش حقیقی
        Buy_N_Volume = Driver.find_element_by_css_selector('#e3 > div:nth-child(1)')
        Buy_N_Volume = Change_To_Num(Buy_N_Volume)
        print('حجم فروش حقیقی ---> ',Buy_N_Volume)
        #e4 > div:nth-child(1)  حجم فروش حقوقی
        Sell_N_Volume = Driver.find_element_by_css_selector('#e4 > div:nth-child(1)')
        Sell_N_Volume = Change_To_Num(Sell_N_Volume)
        print('حجم فروش حقوقی ---> ',Sell_N_Volume)
        #e5    تعداد خرید حقیقی
        Buy_CountI = Driver.find_element_by_css_selector('#e5')
        Buy_CountI = Change_To_Num(Buy_CountI)
        print('تعداد خرید حقیقی ---> ',Buy_CountI)
        #e8    تعداد فروش حقیقی
        Sell_CountI = Driver.find_element_by_css_selector('#e8')
        Sell_CountI = Change_To_Num(Sell_CountI)
        print('تعداد فروش حقیقی ---> ',Sell_CountI)
        #e6    تعداد خرید حقوقی
        Buy_CountN = Driver.find_element_by_css_selector('#e6')
        Buy_CountN = Change_To_Num(Buy_CountN)
        print('تعداد خرید حقوقی ---> ',Buy_CountN)
        #e9    تعداد فروش حقوقی
        Sell_CountN = Driver.find_element_by_css_selector('#e9')
        Sell_CountN = Change_To_Num(Sell_CountN)
        print('تعداد فروش حقوقی ---> ',Sell_CountN)
    except Exception as E:
        print('Exception : ',E)
#         pass
# #         continue

MainBox --->  سرمايه‌گذاري‌ صنعت‌ نفت‌
Price --->  14100
Changes --->  20
Percent_Change --->  0.0014000000000000002
Exception :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"#d03 > span:nth-child(2)"}
  (Session info: chrome=96.0.4664.110)

MainBox --->  پالايش نفت اصفهان
Price --->  6320
Changes --->  130
Percent_Change --->  0.021
Exception :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"#d03 > span:nth-child(2)"}
  (Session info: chrome=96.0.4664.110)

MainBox --->  تامين سرمايه اميد
Price --->  8690
Changes --->  40
Percent_Change --->  0.0046
Exception :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"#d03 > span:nth-child(2)"}
  (Session info: chrome=96.0.4664.110)

MainBox --->  ملي‌ صنايع‌ مس‌ ايران‌
Price --->  7160
Changes --->  150
Percent_Change --->  0.021400000000000002
Exception :  Message: no such element: Unable to locate element: {"method"